In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from scipy.stats import truncnorm
import torch.nn.init as init
import tensorflow as tf
import numpy as np
import pickle as pkl
torch.backends.cudnn.enabled = False

In [2]:
# w0_enc = tf.get_variable("gaussian_MLP_encoder/w0", shape=[784, 500])
# b0_enc = tf.get_variable("gaussian_MLP_encoder/b0", shape=[500])
# w1_enc = tf.get_variable("gaussian_MLP_encoder/w1", shape=[500, 500])
# b1_enc = tf.get_variable("gaussian_MLP_encoder/b1", shape=[500])
# w2_enc = tf.get_variable("gaussian_MLP_encoder/w2", shape=[500, 40])
# b2_enc = tf.get_variable("gaussian_MLP_encoder/b2", shape=[40])


# w0_dec = tf.get_variable("bernoulli_MLP_decoder/w0", shape=[20, 500])
# b0_dec = tf.get_variable("bernoulli_MLP_decoder/b0", shape=[500])
# w1_dec = tf.get_variable("bernoulli_MLP_decoder/w1", shape=[500, 500])
# b1_dec = tf.get_variable("bernoulli_MLP_decoder/b1", shape=[500])
# w2_dec = tf.get_variable("bernoulli_MLP_decoder/w2", shape=[500, 784])
# b2_dec = tf.get_variable("bernoulli_MLP_decoder/b2", shape=[784])

# saver = tf.train.Saver()

In [3]:
# with tf.Session() as sess:
# #   chkp.print_tensors_in_checkpoint_file("/home/iistlab/Desktop/daal_code-master/models/model_class_10_dim_20.ckpt", tensor_name='bernoulli_MLP_decoder/b0', all_tensors=True)
#     saver.restore(sess, "/home/iistlab/Desktop/daal_code-master/models/model_class_10_dim_20.ckpt")
#     w0_encoder = nn.Parameter(torch.transpose(torch.Tensor(w0_enc.eval()),1,0))
#     b0_encoder = nn.Parameter(torch.Tensor(b0_enc.eval()))
#     w1_encoder = nn.Parameter(torch.transpose(torch.Tensor(w1_enc.eval()),1,0))
#     b1_encoder = nn.Parameter(torch.Tensor(b1_enc.eval()))
#     w2_encoder = nn.Parameter(torch.transpose(torch.Tensor(w2_enc.eval()),1,0))
#     b2_encoder = nn.Parameter(torch.Tensor(b2_enc.eval()))
    
#     w0_decoder = nn.Parameter(torch.transpose(torch.Tensor(w0_dec.eval()),1,0))
#     b0_decoder = nn.Parameter(torch.Tensor(b0_dec.eval()))
#     w1_decoder = nn.Parameter(torch.transpose(torch.Tensor(w1_dec.eval()),1,0))
#     b1_decoder = nn.Parameter(torch.Tensor(b1_dec.eval()))
#     w2_decoder = nn.Parameter(torch.transpose(torch.Tensor(w2_dec.eval()),1,0))
#     b2_decoder = nn.Parameter(torch.Tensor(b2_dec.eval()))
    



In [4]:
batch_size = 128
num_epochs = 200

n_hidden = 500 

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size= batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size= batch_size, shuffle=False)

In [7]:
# def kaiming_init(m):
#     if isinstance(m, (nn.Linear)):
#         init.kaiming_uniform(m.weight)
#         if m.bias is not None:
#             m.bias.data.fill_(0.)

In [8]:


class VAE(nn.Module):
    def __init__(self, nz):
        super(VAE, self).__init__()

        self.have_cuda = False
        self.nz = nz

        self.fc1 = nn.Linear(784, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, self.nz * 2)
        
        self.fc1.weight = w0_encoder
        self.fc1.bias = b0_encoder
        
        self.fc2.weight = w1_encoder
        self.fc2.bias = b1_encoder
            
        self.fc3.weight = w2_encoder
        self.fc3.bias = b2_encoder
        
    
        self.fc4 = nn.Linear(nz,  n_hidden)
        self.fc5 = nn.Linear(n_hidden, n_hidden)
        self.fc6 = nn.Linear(n_hidden, 784)
            
        self.fc4.weight = w0_decoder
        self.fc4.bias = b0_decoder
        
        self.fc5.weight = w1_decoder
        self.fc5.bias = b1_decoder
        
        self.fc6.weight = w2_decoder
        self.fc6.bias = b2_decoder
        

        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU()
        self.tanh = nn.Tanh()
        self.dropout_encoder = nn.Dropout(0.1)
        self.dropout_decoder = nn.Dropout(0.1)
        
#         self.weight_init()
        
#     def weight_init(self):
#         for block in self._modules:
#             kaiming_init(block)

    def encode(self, x):
        h1 = self.dropout_encoder(self.elu(self.fc1(x)))
        h2 = self.dropout_encoder(self.tanh(self.fc2(h1)))
        gaussian_params = self.fc3(h2)
        mean = gaussian_params[:, :self.nz]
        stddev = 10**-6 + self.softplus(gaussian_params[:,self.nz:])
        return mean, stddev

    def decode(self, z) :
        h1 = self.dropout_decoder(self.tanh(self.fc4(z)))
        h2 = self.dropout_decoder(self.elu(self.fc5(h1)))
        deconv_input = self.sigmoid(self.fc6(h2))
        deconv_input = torch.clamp(deconv_input, 10**-8, 1 - 10**-8)
        return deconv_input

    def reparametrize(self, mu, std):
#         std = logvar.mul(0.5).exp()
        eps = torch.cuda.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return mu + std*eps

    def forward(self, x):
        x = x.view(-1,784)
        mu, stddev = self.encode(x)
        # print("mu, logvar", mu.size(), logvar.size())
        z = self.reparametrize(mu, stddev)
        # print("z", z.size())
        decoded = self.decode(z)
        # print("decoded", decoded.size())
        return decoded, mu, stddev


In [9]:
model = VAE(nz=20).to(device)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3)

NameError: name 'w0_encoder' is not defined

In [ ]:
def loss_function(recon_x, x, mu, sigma):
    x  = x.view(-1,784).to(device)
    BCE = - torch.sum(x * torch.log(recon_x) + (1-x) * torch.log(1 - recon_x), 1)
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2),1)
    return BCE.mean(), KLD.mean()

In [ ]:
for epoch in range(num_epochs) :
    train_loss_sum = 0
    model.train()
    for batch_idx, (data, _) in enumerate(train_loader) :
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        bce, kld = loss_function(recon_batch, data, mu, logvar)
        loss = bce + kld
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.data
    train_loss = train_loss_sum / len(train_loader)
    
    test_loss_sum = 0
   
    for batch_idx, (data, _) in enumerate(test_loader) :
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        bce, kld = loss_function(recon_batch, data, mu, logvar)
        loss = bce  + kld
        test_loss_sum += loss.data
    test_loss = test_loss_sum / len(test_loader)
    
    print('-----Number of epochs---',epoch) 
    print('----Training_loss---',train_loss)
    print('----Testing loss----',test_loss)

In [ ]:
z = torch.randn(60000,20)